In [2]:
import re
import graphlab as gl
from graphlab.toolkits.feature_engineering import TFIDF, QuadraticFeatures

# put in the path to the kaggle data
PATH_TO_JSON = "docs_prod_02/"
PATH_TO_TRAIN_LABELS = "input/train.csv"
PATH_TO_TEST_LABELS = "input/sampleSubmission.csv"

gl.canvas.set_target('ipynb')

[WARNING] This Python session does not appear to be running in an interactive IPython Notebook. Use of the 'ipynb' target may behave unexpectedly or result in errors.


In [3]:
sf = gl.SFrame.read_csv(PATH_TO_JSON, header=False)
sf = sf.unpack('X1',column_name_prefix='')

[INFO] This trial license of GraphLab Create is assigned to renatbek@gmail.com and will expire on October 08, 2015. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.

[INFO] Start server at: ipc:///tmp/graphlab_server-5411 - Server binary: /usr/local/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1442309396.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Finished parsing file /mnt/data/docs_prod_02/20
PROGRESS: Parsing completed. Parsed 100 lines in 0.290717 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 2300 lines. Lines per second: 4069.08
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/20
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/7
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/24
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/2
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/0
PROGRESS: Read 33628 lines. Lines per second: 5729.56
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/54
PROGRESS: Finished parsing file /mnt/data/docs_prod_02/51
PROGRESS: Finished parsing file /mnt/data/docs

In [4]:
train_labels = gl.SFrame.read_csv(PATH_TO_TRAIN_LABELS)
test_labels = gl.SFrame.read_csv(PATH_TO_TEST_LABELS)

train_labels['id'] = train_labels['file'].apply(lambda x: str(x.split('_')[0] ))
train_labels = train_labels.remove_column('file')
test_labels['id'] = test_labels['file'].apply(lambda x: str(x.split('_')[0] ))
test_labels = test_labels.remove_column('file')

sf['id'] = sf['id'].apply(lambda x: str(x.split('_')[0] ))

PROGRESS: Finished parsing file /mnt/data/input/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.162445 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /mnt/data/input/train.csv
PROGRESS: Parsing completed. Parsed 337024 lines in 0.139895 secs.
PROGRESS: Finished parsing file /mnt/data/input/sampleSubmission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.054828 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
-------------------------

In [5]:
train = train_labels.join(sf, on='id', how='left')
test = test_labels.join(sf, on='id', how='left')

In [6]:
train.head()

sponsored,id,ahref,atext,img_avg,img_b_avg,img_b_cnt,img_cnt
0,2459274,"[#, #, #, #, #, #, #, /terms_of_use.php?ref=re ...","[register, sign in,continue with facebook, ...",0.0,0.0,0,14
0,2459283,"[http://web-marketing-consultants.com/, http ...","[, home, about us,services, video making, ...",0.0,0.0,0,33
0,2459284,"[http://www.flyfishdiary.com, #, http://www.fl ...","[, , home, equipment, flyfishing, fly tying, n ...",0.0,0.0,0,19
0,2459293,[/article/2947905/security/mozilla-blocks-all- ...,"[mozilla blocks all flashin firefox, 10 most ...",0.0,0.0,0,28
0,245932,[http://abcnews.go.com/International/iran-nucl ...,"[iran nuclear deal, plutonew horizons, el chapo ...",0.0,0.0,0,32
0,2459356,"[http://dailypicdump.com,http://dailypicdump.c ...","[, images, videos, gifs,to ferment and serve, ...",0.0,0.0,0,14
0,2459361,"[//imgur.com, /hot, /new,//imgur.com/blog, ...","[, most viral, usersubmitted, blog, about, ...",0.0,0.0,0,2
0,2459371,"[http://www.schleckysilberstein.com/, http://w ...","[, schlecky silbersteinune source d inspirat ...",0.0,0.0,0,8
0,2459374,"[http://www.sedentario.org/, https://dl.dropbo ...","[sedentrio & hiperativo,, duquian, internet, ...",0.0,0.0,0,58
0,245938,"[http://www.triveditestimonials.com, http://ww ...","[, home, about us, videotestimonial, fear from ...",0.0,0.0,0,11


In [7]:
# a simple method to clean the text within an html response
def create_count_features(sf):
    sf['num_links'] = sf['ahref'].apply(lambda x: len(x))
    sf['num_links_non_short'] = sf['ahref'].apply(lambda x: len([a for a in x if len(a) > 3]))
    sf['num_links_http'] = sf['ahref'].apply(lambda x: len([a for a in x if 'http' in a]))
    sf['num_clean_chars'] = sf['text_clean'].apply(lambda x: len(x))
    sf['num_clean_chars_title'] = sf['title'].apply(lambda x: len(x))
    return sf

def clean_text(sf):
    sf['text_clean'] = sf['par'].apply(lambda x: re.sub(r'[\n\t,.:;()\-\/]+', ' ', ' '.join(x)))
    sf['text_clean'] = sf['text_clean'].apply(lambda x: re.sub(r'\s{2,}', ' ', x))
    sf['text_clean'] = sf['text_clean'].apply(lambda x: x.strip())
    return sf

features = [
            'img_cnt',
            'misc_button', 'misc_div', 'misc_input', 'misc_li', 'misc_meta',
            'script_avg', 'script_cnt', 'script_b_avg', 'script_b_cnt',
            'style_avg', 'style_cnt', 'style_b_avg', 'style_b_cnt',
            'num_links', 'num_links_non_short', 'num_links_http', 'num_clean_chars', 'num_clean_chars_title'
           ]

# a wrapper method around the 2 methods above
def process_dataframe(sf):
    sf = clean_text(sf)
    sf = create_count_features(sf)
    quadratic_sf = gl.feature_engineering.create(sf, QuadraticFeatures(features = features))
    sf = quadratic_sf.transform(sf)
    return sf 

In [8]:
train =  process_dataframe(train)
test = process_dataframe(test)

In [9]:
train.head()

sponsored,id,ahref,atext,img_avg,img_b_avg,img_b_cnt,img_cnt
0,2459274,"[#, #, #, #, #, #, #, /terms_of_use.php?ref=re ...","[register, sign in,continue with facebook, ...",0.0,0.0,0,14
0,2459283,"[http://web-marketing-consultants.com/, http ...","[, home, about us,services, video making, ...",0.0,0.0,0,33
0,2459284,"[http://www.flyfishdiary.com, #, http://www.fl ...","[, , home, equipment, flyfishing, fly tying, n ...",0.0,0.0,0,19
0,2459293,[/article/2947905/security/mozilla-blocks-all- ...,"[mozilla blocks all flashin firefox, 10 most ...",0.0,0.0,0,28
0,245932,[http://abcnews.go.com/International/iran-nucl ...,"[iran nuclear deal, plutonew horizons, el chapo ...",0.0,0.0,0,32
0,2459356,"[http://dailypicdump.com,http://dailypicdump.c ...","[, images, videos, gifs,to ferment and serve, ...",0.0,0.0,0,14
0,2459361,"[//imgur.com, /hot, /new,//imgur.com/blog, ...","[, most viral, usersubmitted, blog, about, ...",0.0,0.0,0,2
0,2459371,"[http://www.schleckysilberstein.com/, http://w ...","[, schlecky silbersteinune source d inspirat ...",0.0,0.0,0,8
0,2459374,"[http://www.sedentario.org/, https://dl.dropbo ...","[sedentrio & hiperativo,, duquian, internet, ...",0.0,0.0,0,58
0,245938,"[http://www.triveditestimonials.com, http://ww ...","[, home, about us, videotestimonial, fear from ...",0.0,0.0,0,11


In [10]:
# create word counts and remove countwords for both train and test
bow_trn = gl.text_analytics.count_words(train['text_clean'])
bow_trn = bow_trn.dict_trim_by_keys(gl.text_analytics.stopwords())

bow_tst = gl.text_analytics.count_words(test['text_clean'])
bow_tst = bow_tst.dict_trim_by_keys(gl.text_analytics.stopwords())

# add the bag of words to both sframes
train['bow'] = bow_trn
test['bow'] = bow_tst

In [ ]:
# create a TFIDF Transformer that is fit on your training data and transform both training and testing data
encoder = gl.feature_engineering.create(train, TFIDF('bow', output_column_name='tfidf'))
train = encoder.transform(train)
test = encoder.transform(test)

In [ ]:
features = features + ['tfidf', 'quadratic_features']

In [ ]:
train.head()

In [ ]:
train = train.dropna()
test = test.dropna()

In [ ]:
train_train, train_cv = train.random_split(0.85, seed=5)

In [ ]:
model = gl.logistic_classifier.create(train_train, target='sponsored', 
                                      features=features,
                                      validation_set=train_cv,
                                      class_weights='auto',
                                      l2_penalty=0.00, l1_penalty=0.0)

In [ ]:
ypred = model.predict(test)

In [ ]:
submission = gl.SFrame()
submission['sponsored'] = ypred 
submission['file'] = test['id'].apply(lambda x: x + '_raw_html.txt')
submission.save('submission_version_3.csv', format='csv')